In [ ]:
import pandas as pd
import geopandas as gpd

## After Expert feedback

In [ ]:
mass_produced = {"uttar_pradesh": 143067, "bihar": 72299, "west_bengal": 57893, "haryana": 22371, "punjab": 17908}
mass_df = pd.DataFrame(mass_produced, index=["mass_produced"]) * 1000000
display(mass_df)

emission_factors = {"CFCBK": {"PM$_{2.5}$": 0.18, "SO$_2$": 0.52, "CO": 3.63, "CO$_2$": 179.00},
                    "FCBK": {"PM$_{2.5}$": 0.18, "SO$_2$": 0.52, "CO": 3.63, "CO$_2$": 179.00},
                    "Zigzag": {"PM$_{2.5}$": 0.09, "SO$_2$": 0.15, "CO": 1.19, "CO$_2$": 107.50}}
emission_df = pd.DataFrame(emission_factors)
display(emission_df)


In [ ]:
states = ["uttar_pradesh", "bihar", "west_bengal", "haryana", "punjab"]
state_to_name = {"uttar_pradesh": "Uttar Pradesh", "bihar": "Bihar", "west_bengal": "West Bengal", "haryana": "Haryana", "punjab": "Punjab", "Total": "Total"}

count_df = pd.DataFrame(columns = ["CFCBK", "FCBK", "Zigzag"])
for state in states:
    gdf = gpd.read_file(f"labels/{state}.geojson")
    CFCBK = len(gdf[gdf['class_name'] == 'CFCBK'])
    FCBK = len(gdf[gdf['class_name'] == 'FCBK'])
    Zigzag = len(gdf[gdf['class_name'] == 'Zigzag'])
    
    count_df.loc[state] = [CFCBK, FCBK, Zigzag]
    
count_df

In [ ]:
# get fractions over classes
count_frac_df = count_df.div(count_df.sum(axis=1), axis=0)
print("Fractions of classes over states")
display(count_frac_df)

# Get weights per state
weights_df = count_frac_df.mul(mass_df.loc["mass_produced"], axis=0)
print("Weights per state")
display(weights_df)

In [ ]:
# Get emissions per state
emissions_kilotonnes_per_day = (weights_df@emission_df.T) / 180 / 1000000
emissions_kilotonnes_per_day['Mass'] = emissions_kilotonnes_per_day.index.map(mass_produced) / 180 * 1000
emissions_kilotonnes_per_day.loc["Total"] = emissions_kilotonnes_per_day.sum(numeric_only=True)
emissions_kilotonnes_per_day.index.name = "State"
emissions_kilotonnes_per_day = emissions_kilotonnes_per_day.reset_index()
emissions_kilotonnes_per_day['State'] = emissions_kilotonnes_per_day['State'].map(state_to_name)
cols = emissions_kilotonnes_per_day.columns.tolist()
emissions_kilotonnes_per_day = emissions_kilotonnes_per_day[[cols[0]] + [cols[-1]] + cols[1:-1]]
emissions_kilotonnes_per_day

In [ ]:
print(emissions_kilotonnes_per_day.to_latex(formatters={"PM$_{2.5}$": "{:.2f}".format, "SO$_2$": "{:.2f}".format, "CO": "{:.2f}".format, "CO$_2$": "{:.2f}".format, "Mass": "{:.2f}"}, index=False).replace("Total &", "\midrule\nTotal &"))

## Old

In [ ]:
brick_weight = 3.21 # kg

production = pd.Series()
production.loc['CFCBK'] = 18000
production.loc['FCBK'] = 18000
production.loc['Zigzag'] = 11000
production

In [ ]:
emission_factors = {"CFCBK": {"PM": 0.89, "SO$_2$": 0.52, "CO": 3.63, "CO$_2$": 179.00},
                    "FCBK": {"PM": 0.89, "SO$_2$": 0.52, "CO": 3.63, "CO$_2$": 179.00},
                    "Zigzag": {"PM": 0.23, "SO$_2$": 0.15, "CO": 1.19, "CO$_2$": 107.50}}

emission_factors = pd.DataFrame(emission_factors).T
emission_factors

In [ ]:
# multiply capacity with factors to get emissions per day
emissions_per_day_per_brick = production.mul(emission_factors.T, axis=0).T * brick_weight
emissions_per_day_per_brick

In [ ]:
states = ["uttar_pradesh", "bihar", "west_bengal", "haryana", "punjab"]
state_to_name = {"uttar_pradesh": "Uttar Pradesh", "bihar": "Bihar", "west_bengal": "West Bengal", "haryana": "Haryana", "punjab": "Punjab"}

count_df = pd.DataFrame(columns = ["CFCBK", "FCBK", "Zigzag"])
for state in states:
    gdf = gpd.read_file(f"../final_data/labels/{state}.geojson")
    CFCBK = len(gdf[gdf['class_name'] == 'CFCBK'])
    FCBK = len(gdf[gdf['class_name'] == 'FCBK'])
    Zigzag = len(gdf[gdf['class_name'] == 'Zigzag'])
    
    count_df.loc[state_to_name[state]] = [CFCBK, FCBK, Zigzag]
    
count_df

In [ ]:
emissions_per_day_per_state = count_df.dot(emissions_per_day_per_brick) / 1000 / 1000
emissions_per_day_per_state.loc['Total'] = emissions_per_day_per_state.sum()
emissions_per_day_per_state.index.name = "State"
emissions_per_day_per_state.reset_index(inplace=True)
print(emissions_per_day_per_state.to_latex(formatters={"PM": "{:.2f}".format, "SO$_2$": "{:.2f}".format, "CO": "{:.2f}".format, "CO$_2$": "{:.2f}".format}, index=False).replace("Total &", "\midrule\nTotal &"))